# **Auto encoders with Bidirectional LSTM**

In [1]:
# Ensure pip is up-to-date
!pip install --upgrade pip

# Install core data science packages
!pip install pandas numpy scikit-learn

# Install TensorFlow and TensorFlow Addons compatible with Python 3.8/3.9
!pip install tensorflow

# Install Keras (integrated with TensorFlow)
!pip install keras

# Install necessary tools for callbacks and optimizers
!pip install tensorflow-probability

# Install Jupyter (if you’re working in a notebook environment)
!pip install notebook ipykernel


  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\Danie\anaconda3\envs\XDS\python.exe -m pip install --upgrade pip


  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.5.2-cp310-cp310-win_amd64.whl.metadata (13 kB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached scikit_learn-1.5.2-cp310-cp310-win_amd64.whl (11.0 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached rich-13.9

  You can safely remove it manually.
  You can safely remove it manually.


  Using cached tensorflow_probability-0.24.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached cloudpickle-3.1.0-py3-none-any.whl.metadata (7.0 kB)
Using cached tensorflow_probability-0.24.0-py2.py3-none-any.whl (6.9 MB)
Using cached cloudpickle-3.1.0-py3-none-any.whl (22 kB)


In [5]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)


In [12]:
!pip install tensorflow-addons==0.16.1

  Using cached tensorflow_addons-0.16.1-cp310-cp310-win_amd64.whl.metadata (1.8 kB)
Using cached tensorflow_addons-0.16.1-cp310-cp310-win_amd64.whl (755 kB)
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 0.22.0
    Uninstalling tensorflow-addons-0.22.0:
      Successfully uninstalled tensorflow-addons-0.22.0


In [13]:
# from google.colab import files
# uploaded = files.upload()

In [10]:
pip install protobuf==3.20.*


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow==2.8.0 tensorflow-addons==0.16.1


Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [6]:
import pandas as pd
import io
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa

from keras import Sequential
from keras import layers
from keras.models import Model
from keras.layers import LSTM, Bidirectional,RepeatVector, BatchNormalization, Dropout, Dense, Flatten, Conv1D
from keras.layers import MaxPooling1D, GRU, Input,Masking, Concatenate, dot
from tensorflow.keras.optimizers import Adam, SGD
from keras.losses import MeanAbsoluteError
from keras.metrics import RootMeanSquaredError
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
#from tensorflow.keras.optimizers import legacy
from sklearn import metrics
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_excel('UpdatedDataSet.xlsx')
df.shape

(70128, 76)

In [8]:
df.head()

,Feat 1,Feat 2,Feat 3,Feat 4,Feat 5,Feat 6,Feat 7,Feat 8,Feat 9,Feat 10,...,Feat 67,Feat 68,Feat 69,Feat 70,Feat 71,Feat 72,Feat 73,Feat 74,Feat 75,Target
0,17.01,7.67,14.67,7.16,6.33,14.59,14.95,8.09,14.73,14.70,...,33.0,-205.0,3238.0,1938.0,5423.0,0.0,3566.0,421.0,313.0,24.68
1,67.11,-15.25,42.65,-16.14,-25.31,42.00,41.76,-13.63,43.07,42.58,...,36.0,-188.0,2860.0,1390.0,5423.0,0.0,3578.0,397.0,310.0,63.18
2,71.15,0.54,50.97,-0.88,-8.37,50.40,50.75,2.48,51.33,50.99,...,38.0,-63.0,2721.0,1251.0,5423.0,0.0,2340.0,449.0,310.0,66.96
3,286.20,8.53,206.96,2.49,-27.06,204.83,206.61,17.17,208.39,207.09,...,29.0,-68.0,2713.0,1193.0,5422.0,0.0,2238.0,442.0,311.0,21.42
4,17.53,0.98,12.82,0.61,-1.13,12.70,12.80,1.56,12.91,12.83,...,33.0,-68.0,2127.0,1192.0,5423.0,0.0,1905.0,466.0,305.0,21.47


In [9]:
y = df.Target - min(df.Target) +1
y= np.log1p(y)
x = df.drop("Target", axis =1)

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [23]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(56102, 75)
(56102,)
(14026, 75)
(14026,)


In [24]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.125, random_state=42)

In [25]:
input_shape = (75, 1)
encoding_dim = 19

In [26]:
# Encoder
inputs = Input(shape=input_shape)
encoded = Bidirectional(LSTM(encoding_dim, activation='relu'))(inputs)
print(encoded)

KerasTensor(type_spec=TensorSpec(shape=(None, 38), dtype=tf.float32, name=None), name='bidirectional_2/concat:0', description="created by layer 'bidirectional_2'")


In [27]:
inputs

<KerasTensor: shape=(None, 75, 1) dtype=float32 (created by layer 'input_2')>

In [28]:
# Decoder
decoded = RepeatVector(input_shape[0])(encoded)
decoded = Bidirectional(LSTM(1, return_sequences=True, activation='relu'))(decoded)

In [29]:
autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

In [30]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

In [31]:
x_train = x_train.reshape((-1, x_train.shape[1], 1))
x_val = x_val.reshape((-1, x_val.shape[1], 1))

In [32]:
# Train the autoencoder first
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train, x_train, epochs=100, batch_size=64, validation_split = 0.125, callbacks = [callback])

Epoch 1/100
839/839 [==============================] - 81s 90ms/step - loss: 1.2607 - val_loss: 0.9119
Epoch 2/100
839/839 [==============================] - 74s 89ms/step - loss: 0.9394 - val_loss: 0.8508
Epoch 3/100
839/839 [==============================] - 75s 90ms/step - loss: 0.9037 - val_loss: 0.8452
Epoch 4/100
839/839 [==============================] - 74s 88ms/step - loss: 0.8973 - val_loss: 0.8464
Epoch 5/100
839/839 [==============================] - 74s 88ms/step - loss: 0.8889 - val_loss: 0.8421
Epoch 6/100
 74/839 [=>............................] - ETA: 1:06 - loss: 0.8131

KeyboardInterrupt: 

In [33]:
# x_train = x_train.reshape((-1, x_train.shape[1], 1))
# x_val = x_val.reshape((-1, x_val.shape[1], 1))
x_train.shape

(61362, 75, 1)

In [34]:

# Then use the encoder to transform data
x_train_encoded = encoder.predict(x_train)
x_val_encoded = encoder.predict(x_val)
print(x_train_encoded.shape)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(61362, 38)


In [35]:
print(x_train[6:10])

[[[-1.26739975e-01]
  [ 3.23676151e-02]
  [-9.50721866e-02]
  [ 4.83965914e-02]
  [ 1.21049330e-01]
  [-9.03212518e-02]
  [-1.41115845e-01]
  [ 4.25396067e-02]
  [-9.68499828e-02]
  [-1.11211853e-01]
  [ 1.16580328e-01]
  [-1.39629261e-01]
  [ 5.94396677e-02]
  [-2.88545613e-02]
  [-8.11009926e-03]
  [-1.67297043e-01]
  [-8.58417990e-02]
  [-1.40882665e-01]
  [-1.74112787e-01]
  [-2.17291489e-01]
  [-2.81828244e-01]
  [-2.76185592e-01]
  [-3.06299318e-01]
  [-3.06444611e-01]
  [-3.12525000e-01]
  [-3.18306276e-01]
  [-2.88035491e-01]
  [-2.49753138e-01]
  [-2.49008962e-01]
  [-2.20224529e-01]
  [-1.79866408e-01]
  [-1.53900756e-01]
  [-1.71405410e-01]
  [-1.39355792e-01]
  [-1.56713393e-01]
  [-1.70658277e-01]
  [-1.73478440e-01]
  [-1.78225877e-01]
  [-1.84150367e-01]
  [-1.90706482e-02]
  [ 3.14901576e-01]
  [ 7.56659038e-02]
  [ 3.17575946e+00]
  [-1.50130678e-01]
  [-2.03579480e-01]
  [ 3.89406770e-01]
  [ 3.56956942e-02]
  [ 1.82601640e-01]
  [ 1.41344311e-01]
  [ 8.70497744e-02]


In [23]:
x_train_encoded_flattened = x_train_encoded.flatten()
#x_train_diff = pd.DataFrame({'Actual value': x_train.flatten(), 'Predicted value': x_train_encoded_flattened})
print(x_train_encoded[6:10])

[[7.55288422e-01 9.68944150e-05 4.49130028e-01 7.93407007e-11
  1.56900734e-01 8.24337378e-02 1.45068273e-01 4.12939303e-03
  1.89912766e-01 1.11747300e-02 1.31299146e-20 0.00000000e+00
  2.54911959e-01 7.00145289e-02 1.58209711e-01 4.19519752e-01
  7.33837485e-01 8.45711827e-01 1.68193564e-01 3.52143953e-10
  9.26109759e-17 1.02022417e-01 0.00000000e+00 2.71651804e-01
  3.81069682e-08 0.00000000e+00 8.82040784e-02 2.14828047e-13
  3.88590449e-09 1.48034796e-01 6.46805347e-08 5.38558625e-02
  4.36979562e-01 1.17758640e-13 1.26312047e-01 0.00000000e+00
  1.47108212e-01 3.19248259e-01]
 [3.96776652e+00 1.95584523e-17 5.72962880e-01 1.50741090e-03
  3.90163809e-01 5.95271774e-02 3.28171328e-02 3.89283746e-02
  3.22302245e-02 1.32400664e-27 2.09941348e-21 0.00000000e+00
  8.40954900e-01 7.04610255e-03 2.17183426e-01 1.95078226e-16
  7.09952772e-01 3.62102151e-01 8.07056644e-10 3.67596442e-09
  8.79134823e-05 2.12388515e-01 3.54006363e-04 2.47221082e-01
  8.63179666e-05 7.25660296e-11 1.533

In [36]:
# Encoded input
encoded_input = Input(shape=(38,))
x = RepeatVector(75)(encoded_input)
x = Bidirectional(LSTM(units=100, return_sequences=True, activation="relu", recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))(x)
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(0.1)(x)
predictions = Dense(1)(x)

combined_model = Model(encoded_input, predictions)


In [37]:
epoch = 100
batch_size = 64
# Define cyclic learning rate, optimizer, and callback
steps_per_epoch = len(x_train) // batch_size
cyclic_lr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=1e-04,
                                                maximal_learning_rate=1e-02,
                                                scale_fn=lambda x: 1/(2**(x-1)),
                                                step_size=6 * steps_per_epoch)
optimizer = Adam(learning_rate=cyclic_lr, amsgrad=True)


# Compile and train the combined model
combined_model.compile(optimizer=optimizer, loss='mean_absolute_error')
combined_model.fit(x_train_encoded, y_train, validation_data=(x_val_encoded, y_val), epochs=epoch, batch_size=batch_size, callbacks=[callback])

Epoch 1/100
959/959 [==============================] - 262s 267ms/step - loss: 28318.9609 - val_loss: 109.6208
Epoch 2/100
959/959 [==============================] - 252s 263ms/step - loss: 18.0043 - val_loss: 4.2725
Epoch 3/100
173/959 [====>.........................] - ETA: 3:25 - loss: 4.8894

KeyboardInterrupt: 

In [38]:
x_test = x_test.reshape((-1, x_test.shape[1], 1))
# Use encoder to transform x_test and make predictions
x_test_encoded = encoder.predict(x_test)
y_predict = combined_model.predict(x_test_encoded)

y_predict = y_predict.flatten()
print(y_predict)
diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_predict})
print(diff)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[6.8642755 6.9025826 4.7457647 ... 6.866116  6.75458   6.925442 ]
       Actual value  Predicted value
32189      7.002256         6.864275
41315      7.005880         6.902583
57975      7.071106         4.745765
14401      7.018026         6.920862
8550       7.011998         7.045215
...             ...       

In [39]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [40]:
meanAbErr = metrics.mean_absolute_error(y_test, y_predict)
meanSqErr = metrics.mean_squared_error(y_test, y_predict)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_predict))
mape= MAPE(y_test,y_predict)
#print('R squared: {:.2f}'.format(regr.score(x,y)*100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Absolute percentage Error:', mape)
#print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

Mean Absolute Error: 0.27376475370671693
Mean Absolute percentage Error: 3.864623589888421
Root Mean Square Error: 1.768812374224809
